In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
import pickle as pkl

In [3]:
# get html code from website
def getSoup(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    return soup

In [4]:
# gets body of stories (with some extra noise) - edit accordingly depending on htmls from each site
def getStories(stories, start):
    newStories = []
    currentStory = ''
    for story in stories:
        if story == stories[0]:
            newStories.append(story)
        elif '<p>' + start in story:
            newStories.append(currentStory)
            currentStory = story
        elif story == stories[-1]:
            newStories.append(currentStory)
        else:
            currentStory += story
    return newStories

def getStories2(stories):
    newStories = []
    currentStory = []
    for num, story in enumerate(stories):
        if 'br/' in story or story == '<p></p>':
            newStories.append(currentStory)
            currentStory = [story]
        elif story == stories[-1]:
            currentStory.append(story)
            newStories.append(currentStory)
        else:
            currentStory.append(story)
    return newStories

def getStories3(stories, end):
    newStories = []
    currentStory = ''
    for story in stories:
        if end + '</p>' in story:
            currentStory += story
            newStories.append(currentStory)
            currentStory = ''
        elif story == stories[-1]:
            newStories.append(currentStory)
        else:
            currentStory += story
    return newStories

def getStories4(stories):
    newStories = []
    counter = 2
    currentStory = ''
    for story in stories:
        if story == stories[0]:
            currentStory += story
        elif '>' + str(counter) + '.' in story:
            newStories.append(currentStory)
            currentStory = story
            counter += 1
        elif story == stories[-1]:
            newStories.append(currentStory)
        else:
            currentStory += story
    return newStories

In [5]:
# turn list of lists of stories into list of strings of stories
def addStories(newStories):
    myStories = []
    for story in newStories:
        currentStory = ''
        for item in story:
            currentStory += item
        myStories.append(currentStory)
    return myStories

In [6]:
# find common factor in stories which defines beginning and end and make stories only that (get rid of outside noise)
def startToEndStories(newStories, start, end):    
    scaryStories = []
    for index, story in enumerate(newStories):
        startIndex = newStories[index].find(start)
        endIndex = newStories[index].find(end, startIndex + 1)
        scaryStories.append(newStories[index][startIndex + 1:endIndex])
    return scaryStories

def startToEndStories2(newStories, start):    
    scaryStories = []
    for index, story in enumerate(newStories):
        startIndex = newStories[index].find(str(start))
        scaryStories.append(newStories[index][startIndex + 2:-4])
        start += 1
    return scaryStories

In [7]:
# removes html paragraph tags to completely clean stories - more might need to be done before this step (get rid of inside noise)
def cleanStories(scaryStories):
    cleanStories = []
    for story in scaryStories:
        if len(story) > 1:
            cleanStories.append(story.replace('<p>','').replace('</p>', ''))
    return cleanStories

# 67 True Scary Stories

In [8]:
# get soup for 67 story site
soup = getSoup('https://thoughtcatalog.com/erin-cossetta/2017/10/true-scary-stories-to-tell-in-the-dark/')

# get lines that contain stories
stories = [str(story) for story in soup.find_all('p')]

# get characters that distinguish beginning and end of stories
weirdQuote = stories[0][3]
weirdEndQuote = stories[0][-153]
dash = stories[0][565:568]

# clean stories
newStories = getStories(stories, weirdQuote)
newStories = newStories[:8] + [str(newStories[8] + newStories[9] + newStories[10] + newStories[11] + newStories[12] + newStories[13])] + newStories[14:]
scaryStories = startToEndStories(newStories, weirdQuote, dash)
cleanStories67 = cleanStories(scaryStories)

In [9]:
cleanStories67[0]

'Making toast late one night facing the kitchen bench eating for a good 5-10mins. Cleaned up and turned around and every single cupboard door and cutlery drawer were open. They were all closed when I went into the kitchen, and there was no way anyone could have snuck in and done it because I was home alone. I also didn’t hear them open in the time I was there. Scared the shit out of me so I froze, let it register for a few seconds, then calmly closed everything and went back to my room to hide until daylight. One of a few weird things to happen growing up.'

# 50 Scary Stories

In [10]:
# get soup for 50 story site
soup = getSoup('https://thoughtcatalog.com/christine-stockton/2018/09/50-scary-stories-that-turned-out-to-be-100-real/')

# get lines that contain stories
stories = stories = [str(story) for story in soup.find_all('p') if "href" not in str(story)]

# get characters that distinguish beginning and end of stories
weirdQuote = stories[1][3]
weirdEndQuote = stories[3][-5]

# clean stories
newStories = getStories2(stories)
myStories = addStories(newStories)
scaryStories = startToEndStories(myStories, weirdQuote, weirdEndQuote + '</p>')
cleanStories50 = cleanStories(scaryStories)

In [11]:
cleanStories50[1]

'A guy I know is a boat mechanic. So one Saturday he’s on his own doing some work on a boat and suddenly he comes too and he’s on the ground. He knew there was water under the boat he’d been working on so he assumes he’s been electrocuted and he’s vaguely aware of someone behind him. So he starts yelling “help help I’ve been electrocuted” and he hears whoever is behind him leave. He keeps yelling and eventually a guy in a neighbouring business hears him and comes to his aid. When he’s at the hospital he finds out he wasn’t electrocuted, someone had hit him on the back of the head with his own mallet. Nothing was stolen from the shop.Meaning someone (who was never caught) was just walking by and spontaneously decided to hit him on the back of the head for no reason.'

# 100 Glitch In the Matrix

In [14]:
# get soup for 100 story site
soup = getSoup('https://thoughtcatalog.com/juliet-lanka/2017/11/25-people-give-their-glitch-in-the-matrix-story-that-made-them-believe-in-the-supernatural/')

# get lines that contain stories
stories = [str(story) for story in soup.find_all('p')][1:]

# get characters that distinguish beginning and end of stories
weirdQuote = stories[0][3]
weirdEndQuote = stories[2][-5]

# clean stories
newStories = getStories3(stories, weirdEndQuote)
scaryStories = startToEndStories(newStories, weirdQuote, weirdEndQuote + '</p>')
newScaryStories = []
for story in stories:
    newStory = story
    count = newStory.count('<')
    if count != 0:
        for x in range(count):
            startIndex = newStory.find('<')
            endIndex = newStory.find('>', startIndex + 1)
            newStory = newStory[:startIndex] + newStory[endIndex+1:]
    if '\n' in newStory:
        newStory = newStory.replace('\n','')
    if '\xa0' in newStory:
        newStory = newStory.replace('\xa0','')
    newScaryStories.append(newStory)
cleanStories100 = cleanStories(newScaryStories)

In [17]:
cleanStories100[5]

'“My friend and I were talking to our room mate and her girlfriend one night. Room mate had just received her new phone and was sitting in bed using it while the four of us chatted.So, room mate had sat her phone down while we were all chatting. She goes to pick it back up and it’s gone! Like, she had just sat it down for a few minutes tops before she noticed that the thing has poofed. I call it for her and we can’t hear it. We take the comforter and sheets off the bed and shake them all out. We look under the bed, behind it, EVERYWHERE we can. That phone is GONE. Even though we hadn’t left her room at all during our conversation when the phone disappeared, we tore apart the whole apartment trying to find it.Finally, we decide to try her car.It was in the locked trunk.We lived on the third floor and were all in the same room watching her use it before it teleported.”'

# 17 Creepy Driving At Night Stories

In [18]:
# get soup for 17 story site
soup17 = getSoup('https://thoughtcatalog.com/erin-cossetta/2017/10/17-creepy-driving-at-night-stories-that-will-freak-you-out-every-time-you-get-in-your-car/')

# get lines that contain stories
stories = [str(story) for story in soup17.find_all('p')][1:]

# get characters that distinguish beginning and end of stories
weirdQuote = stories[0][3]
weirdEndQuote = stories[0][-5]

# clean stories
newStories = getStories(stories, weirdQuote)
scaryStories = startToEndStories(newStories, weirdQuote, weirdEndQuote + '</p>')
cleanStories17 = cleanStories(scaryStories)

In [29]:
cleanStories17[10]

'I was driving home from work about 2 am, it was a beautiful night so I decided to take a route home that was more in the country instead of through town. I had to cross an old bridge that had been there probably 80-100 years. There’s no houses or lights except my head lights. Just as my front tires hit the bridge a red fog rolls in across it. The hair stood up on the back of my neck and I got across that bridge as fast as possible. After I got across I looked in my mirror and the fog is gone but it looked like someone was standing in the middle of the bridge. I mentioned it to my parents the next morning and my dad said he’d always heard stories about that bridge and supposedly a young lady hung herself from it.'

# 20 Ouija Board Stories

In [30]:
# get soup for 20 story site
soup20 = getSoup('https://thoughtcatalog.com/christine-stockton/2017/06/ouija-board-stories/')

# get lines that contain stories
stories = [str(story) for story in soup20.find_all('p')]

# clean stories
newStories = getStories4(stories)
scaryStories = startToEndStories2(newStories, 1)
cleanStories20 = []
for story in scaryStories:
    while '<' in story:
        startIndex = story.find('<')
        endIndex = story.find('>', startIndex + 1)
        story = story[:startIndex] + story[endIndex+1:]
    if '>' in story:
        index = story.find('>')
        story = story[index+4:]
    story = story.replace('\xa0', ' ')
    if story[0] == ' ':
        story = story[1:]
    elif story[0:2] == '. ':
        story = story[2:]
    cleanStories20.append(story)

In [36]:
cleanStories20[5]

'Choked: I have no evidence, and I do not care if you don’t believe me. I’ve used a board with results, and let something into my home. And have been physically assaulted by this entity.It started out with that feeling like you’re being watched, and doors closing, and footsteps on the hardwood when you were home alone. And progressed slowly into being kept awake by something shaking the bed, or pulling off your covers. Sometimes even whispering your name. The board would disappear for days on end, then show up in places you never would have put it. I became obsessed with it. Then it was a black mass in the corner of the room. Or the silhouette of a man watching you from the doorway. After that it escalated pretty quickly. I had my hair pulled. Fingers pricked. Scratched. Choked. Held down in bed while this thing whispered in my ear in what could have only been Latin….. We had our house blessed and the bad thing hasn’t shown back up. Just the normal occurrences now. But I will never aga

# 42 Creepy Kids Stories

In [37]:
# clean stories for 42 website - different than all the others
cleanStories42 = []
for x in range(1,11):
    if x == 1:
        url = 'https://thoughtcatalog.com/eric-redding/2016/05/42-creepy-af-things-kids-said-to-their-parents-that-absolutely-chilled-them-to-the-bone/'
    else:
        url = 'https://thoughtcatalog.com/eric-redding/2016/05/42-creepy-af-things-kids-said-to-their-parents-that-absolutely-chilled-them-to-the-bone/{}/'
        url = url.format(str(x))
        
    soup42 = getSoup(url)
    stories = [str(story) for story in soup42.find_all('blockquote')]
    for story in stories:
        while '<' in story:
            startIndex = story.find('<')
            endIndex = story.find('>', startIndex + 1)
            story = story[:startIndex] + story[endIndex+1:]
        story = story.replace('\n', '')
        cleanStories42.append(story)

In [42]:
cleanStories42[4]

'I was on a bus recently and we were stopped outside a walk-in clinic. A little girl in the seat in front of me turned to her dad and said, “Death is the poor man’s doctor.” And that was that.'

# Clean and Pickle Stories By Website For Recommender Filter

In [56]:
# clean story function
def finalClean(stories):
    scaryStories = []
    for story in stories:
        newStory = story
        count = newStory.count('<')
        if count != 0:
            for x in range(count):
                startIndex = newStory.find('<')
                endIndex = newStory.find('>', startIndex + 1)
                newStory = newStory[:startIndex] + newStory[endIndex+1:]
        if '\n' in newStory:
            newStory = newStory.replace('\n','')
        if '\xa0' in newStory:
            newStory = newStory.replace('\xa0','')
        if story not in scaryStories:
            scaryStories.append(newStory)
    return scaryStories

In [57]:
# apply final clean to stories from each site
cleanStories100 = finalClean(cleanStories100)

cleanStories17 = finalClean(cleanStories17)

cleanStories20 = finalClean(cleanStories20)

cleanStories42 = finalClean(cleanStories42)

cleanStories50 = finalClean(cleanStories50)

cleanStories67 = finalClean(cleanStories67)

In [58]:
# put all stories in one list
stories = [cleanStories100, cleanStories17, cleanStories20, cleanStories42, cleanStories50, cleanStories67]

In [59]:
# write cleaned stories to picklefile
with open('ScaryStories', 'wb') as picklefile:
    pkl.dump(stories, picklefile)